In [1]:
# STEP 1: IMPORT LIBRARIES
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
import holidays
import warnings
warnings.filterwarnings("ignore")

In [2]:
# STEP 2: LOAD DATA
train = pd.read_csv("train.csv", parse_dates=['doj'])
test = pd.read_csv("/content/test_8gqdJqH.csv", parse_dates=['doj'])
transactions = pd.read_csv("transactions.csv", parse_dates=['doj', 'doi'])

# STEP 3: FILTER TRANSACTIONS FOR DBD=15
trans_15 = transactions[transactions['dbd'] == 15].copy()

# STEP 4: MERGE TRAIN & TEST WITH TRANS_15
train_merged = pd.merge(train, trans_15, on=['doj', 'srcid', 'destid'], how='left')
test_merged = pd.merge(test, trans_15, on=['doj', 'srcid', 'destid'], how='left')


In [3]:
# STEP 5: FEATURE ENGINEERING FUNCTION
india_holidays = holidays.India()
def create_features(df):
    df['day_of_week'] = df['doj'].dt.weekday
    df['month'] = df['doj'].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['route'] = df['srcid'].astype(str) + '_' + df['destid'].astype(str)
    df['search_to_book_ratio'] = df['cumsum_searchcount'] / (df['cumsum_seatcount'] + 1)
    df['search_to_book_ratio'] = df['search_to_book_ratio'].clip(upper=100)
    # Encode region and tier
    for col in ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
    # Add holiday feature
    df['is_holiday'] = df['doj'].isin(india_holidays).astype(int)
    # Add school vacation (May-June, Dec-Jan in most regions)
    df['is_school_vacation'] = df['month'].isin([5, 6, 12, 1]).astype(int)
    # Add wedding season indicator (Feb, Nov, Dec are common months)
    df['is_wedding_season'] = df['month'].isin([2, 11, 12]).astype(int)
    return df

train_feat = create_features(train_merged.copy())
test_feat = create_features(test_merged.copy())

In [4]:
# STEP 6: DEFINE FEATURES & TARGET
features = [
    'day_of_week', 'month', 'is_weekend', 'is_holiday',
    'is_school_vacation', 'is_wedding_season',
    'srcid', 'destid', 'srcid_region', 'destid_region',
    'srcid_tier', 'destid_tier',
    'cumsum_seatcount', 'cumsum_searchcount', 'search_to_book_ratio'
]
target = 'final_seatcount'

# Fill missing values
for df in [train_feat, test_feat]:
    df[features] = df[features].fillna(-1)

X = train_feat[features]
y = train_feat[target]
X_test = test_feat[features]

In [6]:
# STEP 7: STACKING SETUP
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Out-of-fold and test predictions
oof_lgb = np.zeros(len(X))
oof_xgb = np.zeros(len(X))
oof_ridge = np.zeros(len(X))

preds_test_lgb = np.zeros(len(X_test))
preds_test_xgb = np.zeros(len(X_test))
preds_test_ridge = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # LightGBM
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    lgb_model = lgb.train({
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.05,
        'verbosity': -1
    },
    train_set=lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_val],
    valid_names=['train', 'valid'])

    oof_lgb[val_idx] = lgb_model.predict(X_val)
    preds_test_lgb += lgb_model.predict(X_test) / n_splits

    # XGBoost
    xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05)
    xgb_model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  verbose=False)
    oof_xgb[val_idx] = xgb_model.predict(X_val)
    preds_test_xgb += xgb_model.predict(X_test) / n_splits

    # Ridge
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train, y_train)
    oof_ridge[val_idx] = ridge_model.predict(X_val)
    preds_test_ridge += ridge_model.predict(X_test) / n_splits

    # Print fold RMSEs
    rmse_lgb = mean_squared_error(y_val, oof_lgb[val_idx])**0.5
    rmse_xgb = mean_squared_error(y_val, oof_xgb[val_idx])**0.5
    rmse_ridge = mean_squared_error(y_val, oof_ridge[val_idx])**0.5
    print(f"Fold {fold+1} RMSE - LGB: {rmse_lgb:.4f}, XGB: {rmse_xgb:.4f}, Ridge: {rmse_ridge:.4f}")

Fold 1 RMSE - LGB: 429.9725, XGB: 441.6310, Ridge: 990.0841
Fold 2 RMSE - LGB: 427.4086, XGB: 437.8318, Ridge: 993.1385
Fold 3 RMSE - LGB: 420.3598, XGB: 428.0485, Ridge: 997.2632
Fold 4 RMSE - LGB: 431.9694, XGB: 439.5836, Ridge: 985.7129
Fold 5 RMSE - LGB: 426.2885, XGB: 435.9157, Ridge: 979.6600


In [7]:
# STEP 8: TRAIN META-MODEL
X_meta = np.vstack([oof_lgb, oof_xgb, oof_ridge]).T
X_meta_test = np.vstack([preds_test_lgb, preds_test_xgb, preds_test_ridge]).T

meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta, y)
preds_final = meta_model.predict(X_meta_test)

from math import sqrt
final_rmse = sqrt(mean_squared_error(y, meta_model.predict(X_meta)))
print(f"\nFinal Ensemble RMSE: {final_rmse:.4f}")


Final Ensemble RMSE: 426.1811


In [8]:
# STEP 9: CREATE SUBMISSION
submission = pd.DataFrame({
    'route_key': test['route_key'],
    'final_seatcount': preds_final.round().astype(int)
})

submission.to_csv("submission.csv", index=False)
print("Submission file saved as 'submission.csv'")

Submission file saved as 'submission.csv'
